In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from lightgbm import LGBMRegressor
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.decoration import ContinuousDomain, CategoricalDomain
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml.preprocessing import PMMLLabelEncoder
from sklearn2pmml.feature_extraction.text import Splitter
from sklearn_pandas import DataFrameMapper

In [82]:
# Dsplay pipelines
from sklearn import set_config
set_config(display="diagram")

In [83]:
import warnings
warnings.filterwarnings('ignore')

In [84]:
df = pd.read_csv("../data/mpg.csv")
df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,chevrolet,malibu,2.4,2008,4,auto(l4),f,22,30,r,midsize
1,volkswagen,new beetle,1.9,1999,4,manual(m5),f,35,44,d,subcompact
2,mercury,mountaineer 4wd,4.0,1999,6,auto(l5),4,14,17,r,suv
3,jeep,grand cherokee 4wd,6.1,2008,8,auto(l5),4,11,14,p,suv
4,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
...,...,...,...,...,...,...,...,...,...,...,...
229,subaru,impreza awd,2.2,1999,4,auto(l4),4,21,26,r,subcompact
230,toyota,corolla,1.8,2008,4,auto(l4),f,26,35,r,compact
231,jeep,grand cherokee 4wd,3.7,2008,6,auto(l5),4,15,19,r,suv
232,land rover,range rover,4.0,1999,8,auto(l4),4,11,15,p,suv


In [85]:
df.dtypes

manufacturer     object
model            object
displ           float64
year              int64
cyl               int64
trans            object
drv              object
cty               int64
hwy               int64
fl               object
class            object
dtype: object

In [86]:
numeric_features = ["displ", "year", "cyl","cty"]
categorical_features = ["manufacturer","trans","drv", "fl","class"]
text_features = ["model"]
date_features = []

In [87]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

mapper = DataFrameMapper(
    [(col_num, [ContinuousDomain()])                                                            for col_num in numeric_features] +
    [(col_cat, [CategoricalDomain(), PMMLLabelEncoder()])                                       for col_cat in categorical_features] +
    [(col_txt, [CategoricalDomain(), CountVectorizer(tokenizer=Splitter(), max_features=5)])   for col_txt in text_features]
)

In [92]:
pipeline = PMMLPipeline([
    ("mapper", mapper),
    ("regressor", LGBMRegressor(n_estimators=1000))
])

In [93]:
display(pipeline)

PMMLPipeline(steps=[('mapper', DataFrameMapper(drop_cols=[],
                features=[('displ', [ContinuousDomain()]),
                          ('year', [ContinuousDomain()]),
                          ('cyl', [ContinuousDomain()]),
                          ('cty', [ContinuousDomain()]),
                          ('manufacturer',
                           [CategoricalDomain(), PMMLLabelEncoder()]),
                          ('trans', [CategoricalDomain(), PMMLLabelEncoder()]),
                          ('drv', [CategoricalDomain(), PMMLLabelEncoder()]),
                          ('fl', [CategoricalDomain(), PMMLLabelEncoder()]),
                          ('class', [CategoricalDomain(), PMMLLabelEncoder()]),
                          ('model',
                           [CategoricalDomain(),
                            CountVectorizer(max_features=5,
                                            tokenizer=<sklearn2pmml.feature_extraction.text.Splitter object at 0x142e65900>)])])),
       ('regressor', LGBMRegressor(n_estimators=1000))])

In [94]:
# use model__sample_weight for weight
pipeline.fit(df, df["hwy"])
df_trans = pipeline[0:1].fit_transform(df)

In [95]:
df_trans

array([[2.400e+00, 2.008e+03, 4.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.900e+00, 1.999e+03, 4.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [4.000e+00, 1.999e+03, 6.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.700e+00, 2.008e+03, 6.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [4.000e+00, 1.999e+03, 8.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [4.600e+00, 2.008e+03, 8.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])